In [1]:
import cv2
import numpy as np

In [2]:
# YOLO object detection
import cv2 as cv
import numpy as np
import time

WHITE = (255, 255, 255)
img = None
img0 = None
outputs = None

# Load names of classes and get random colors
classes = open('coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

# Give the configuration and weight files for the model and load the network.
net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# determine the output layer
ln = net.getLayerNames()
#ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()] #for gpu
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()] #for cpu

In [ ]:
def load_image(path):
    global img, img0, outputs, ln

    img0 = cv.imread(path)
    #print(img0)
    img = img0.copy()

    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time() - t0

    # combine the 3 output groups into 1 (10647, 85)
    # large objects (507, 85)
    # medium objects (2028, 85)
    # small objects (8112, 85)
    outputs = np.vstack(outputs)
    print(outputs)

    post_process(img, outputs, 0.5)
    cv.imshow('window',  img)
    #cv.displayOverlay('window', f'forward propagation time={t:.3}')
    cv.waitKey(0)

def post_process(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            # cv.rectangle(img, p0, p1, WHITE, 1)
        print (boxes, confidences, classIDs)

    indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def trackbar(x):
    global img
    conf = x/100
    img = img0.copy()
    post_process(img, outputs, conf)
    #cv.displayOverlay('window', f'confidence level={conf}')
    cv.imshow('window', img)

cv.namedWindow('window')
cv.createTrackbar('confidence', 'window', 50, 100, trackbar)
#load_image('images/horse.jpg')
#load_image('images/traffic.jpg')
#load_image('images/zoo.jpg')
#load_image('images/kitchen.jpg')
#load_image('images/airport.jpg')
#load_image('images/tennis.jpg')
#load_image('images/wine.jpg')
#load_image('images/bicycle.jpg')
load_image('dataset/img_29er Sailing medemblik #2 /frame51.jpg')
time.sleep(10)

cv.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

[[0.04002061 0.04579944 0.28298363 ... 0.         0.         0.        ]
 [0.04746182 0.03412598 0.22393659 ... 0.         0.         0.        ]
 [0.05321089 0.03310616 0.94457334 ... 0.         0.         0.        ]
 ...
 [0.9894387  0.9902049  0.01862129 ... 0.         0.         0.        ]
 [0.9899972  0.98750633 0.01951872 ... 0.         0.         0.        ]
 [0.9879814  0.9907614  0.16043724 ... 0.         0.         0.        ]]
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] []
[] [] [

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
#Load YOLO Algorithm
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
#To load all objects that have to be detected
classes=[]
with open("coco.names","r") as f:
    read=f.readlines()
for i in range(len(read)):
    classes.append(read[i].strip("\n"))
#Defining layer names
layer_names=net.getLayerNames()
output_layers=[]
for i in net.getUnconnectedOutLayers():
    print(i)
    print(i[0])
    print(i[0]-1)
    output_layers.append(layer_names[i[0]-1])
#Loading the Image
img=cv2.imread("Road.jpg")
height,width,channels=img.shape
#Extracting features to detect objects
blob=cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)
                                                        #Inverting blue with red
                                                        #bgr->rgb
#We need to pass the img_blob to the algorithm
net.setInput(blob)
outs=net.forward(output_layers)
#print(outs)
#Displaying informations on the screen
class_ids=[]
confidences=[]
boxes=[]
for output in outs:
    for detection in output:
        #Detecting confidence in 3 steps
        scores=detection[5:]                #1
        class_id=np.argmax(scores)          #2
        confidence =scores[class_id]        #3
        if confidence >0.5: #Means if the object is detected
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            #Drawing a rectangle
            x=int(center_x-w/2) # top left value
            y=int(center_y-h/2) # top left value
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
           #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
#Removing Double Boxes
indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.3,0.4)
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = classes[class_ids[i]]  # name of the objects
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, label, (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

cv2.imshow("Output",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

200


IndexError: invalid index to scalar variable.

In [8]:
#Load YOLO Algorithms\
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
#To load all objects that have to be detected
classes=[]
with open("coco.names","r") as f:
    read=f.readlines()
for i in range(len(read)):
    classes.append(read[i].strip("\n"))
#Defining layer names
layer_names=net.getLayerNames()
output_layers=[]
for i in net.getUnconnectedOutLayers():
    output_layers.append(layer_names[i[0]-1])

IndexError: invalid index to scalar variable.

In [7]:
print (cv2. __version__)

AttributeError: module 'cv2' has no attribute '__version__'